## How many car crashes occured in Lafayette Park in 2021?

### Initialize workspace

In [3]:
# import libraries
import pandas as pd # for data manipulation
import gspread # to connect with google sheets
from df2gspread import df2gspread as d2g # to interact with google sheets
import geopandas as gpd # for geospatial data manipulation
import os # for file navigation
from datetime import date # for age calculations

/Users/shelbygreen/opt/anaconda3/envs/dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (29,56,57,58,59,63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Import Data

In [46]:
!pip uninstall rtree
# sudo apt install libspatialindex-dev
# pip install rtree

Found existing installation: Rtree 0.9.7
Uninstalling Rtree-0.9.7:
  Would remove:
    /Users/shelbygreen/opt/anaconda3/envs/dev/lib/python3.8/site-packages/Rtree-0.9.7.dist-info/*
    /Users/shelbygreen/opt/anaconda3/envs/dev/lib/python3.8/site-packages/rtree/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [4]:
# create dataframe of crashes in leon county
df = pd.read_csv("../data/semiclean/leon-events.csv")[['report_number', 'crash_year', 'crash_date_time',
                                       'first_harmful_event', 'total_number_of_vehicles', 'total_number_of_persons',
                                       'latitude', 'longitude', 'on_street_road_highway', 'street_address_number',
                                       'from_intersection_of']]

### Filter Data by Year

In [5]:
# select crashes that occured on 1/1/2021 or later
df2021 = df.query(" crash_year >= 2021").reset_index(drop=True)

### QA/QC Check: How many car crashes, in 2021, are missing coordinates?

In [24]:
# QA/QC check -- how many total crashes
# crashes without coordinates

# remove the rows with missing lat/lon data -- these will be addressed in qa/qc
total_crashes = len(df2021)
missing_crashes = len(df2021[df2021['latitude'].isna() | df2021['longitude'].isna()])
print("There were {} total crashes from January to October 2021. {}% are missing coordinates".format(total_crashes, round(100*(missing_crashes/total_crashes), 2)))

There were 4216 total crashes from January to October 2021. 1.59% are missing coordinates


Less than 5% of the data is missing. We'll remove the missing data and then continue with the analysis

### Filter out crashes without coordinates

In [27]:
df2021_clean = df2021[~df2021['latitude'].isna() & ~df2021['longitude'].isna()]

### Filter Data by Neighborhood

In [41]:
# import neighborhood boundary for lafayette park
lafayette_boundary = gpd.read_file('/Users/shelbygreen/Repositories/personal/parcel-inventory/neighborhoods/lafayette-park.json')[['HOA_NAME', 'geometry']]

In [42]:
# convert crashes dataset to geodataframe
gdf_2021 = gpd.GeoDataFrame(df2021_clean, geometry=gpd.points_from_xy(df2021_clean.longitude, df2021_clean.latitude))

# set CRS to WGS84
gdf_2021 = gdf_2021.set_crs("EPSG:4326")


In [45]:
# clip crashes within the bounding box (lafayette park)
test = gpd.clip(gdf_2021, lafayette_boundary)

ImportError: Spatial indexes require either `rtree` or `pygeos`. See installation instructions at https://geopandas.org/install.html

#### Resources:
- https://www.earthdatascience.org/courses/use-data-open-source-python/intro-vector-data-python/vector-data-processing/clip-vector-data-in-python-geopandas-shapely/